##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# データ増強

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/images/data_augmentation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

## 概要

このチュートリアルでは、画像回転などランダムな（しかし現実的な）変換を適用してトレーニングセットの多様性を高める手法である、データ増強について説明します。データ増強の適用方法については、2 つの方法を説明します。1 番目は [Keras 前処理レイヤー](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/)を使用します。2 番目は`tf.image`を使用します。

## セットアップ

In [ ]:
!pip install tf-nightly

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## データセットをダウンロードする

このチュートリアルでは、[tf_flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers) データセットを使用します。便宜上、[TensorFlow Dataset](https://www.tensorflow.org/datasets) を使用してデータセットをダウンロードします。他のデータインポート方法に関する詳細は、[画像読み込み](https://www.tensorflow.org/tutorials/load_data/images)のチュートリアルをご覧ください。


In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

花のデータセットには 5 つのクラスがあります。

In [ ]:
num_classes = metadata.features['label'].num_classes
print(num_classes)

データセットから画像を取得し、それを使用してデータ増強を実演してみましょう。

In [ ]:
get_label_name = metadata.features['label'].int2str

image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

## Keras 前処理レイヤーを使用する

注意: ここで紹介している [Keras 前処理レイヤー](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing)は、現時点では実験的なものです。

### リサイズとリスケール


前処理レイヤーを使用して、画像を一貫性のある形状に[リサイズ](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Resizing)したり、ピクセル値を[リスケール](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling)したりすることができます。

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

注意: 上記のリスケーリングレイヤーは、ピクセル値を`[0,1]`に標準化します。代わりに`[-1,1]`を用いる場合には、`Rescaling(1./127.5, offset=-1)`と書き込みます。


次のようにして、これらのレイヤーを画像に適用した結果を確認します。 

In [ ]:
result = resize_and_rescale(image)
_ = plt.imshow(result)

次のようにして、ピクセルが`[0-1]`であることを確認します。

In [ ]:
print("Min and max pixel values:", result.numpy().min(), result.numpy().max())

### データ増強

データ増強には、前処理レイヤーを使用することも可能です。

前処理レイヤーをいくつか作成し、同じ画像に繰り返して適用してみましょう。

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# Add the image to a batch
image = tf.expand_dims(image, 0)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

`layers.RandomContrast`、`layers.RandomCrop`、`layers.RandomZoom`など、データ増強に使用可能な様々な前処理[レイヤー](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing)が用意されています。

### 前処理レイヤーを使用した 2 つのオプション

これらの前処理レイヤーを使用できる、重要なトレードオフを伴う方法が 2 つあります。

#### オプション 1: 前処理レイヤーをモデルの一部にする

In [ ]:
model = tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # Rest of your model
])

この場合、2 つの重要なポイントがあります。

- データ増強はデバイス上で他のレイヤーと同期して実行されるため、GPU アクセラレーションの恩恵を受けることができます。

- `model.save`を使用してモデルをエクスポートすると、前処理レイヤーはモデルの残りの部分と一緒に保存されます。後でこのモデルをデプロイする場合、画像は自動的に（レイヤーの設定に従い）標準化されます。これにより、サーバーサイドでロジックを再実装する手間が省けます。

注意: データ増強はテスト時には非アクティブなので、（`model.evaluate`や`model.predict`ではなく） `model.fit` への呼び出し時にのみ、入力画像を増強します。

#### オプション 2: 前処理レイヤーをデータセットに適用する

In [ ]:
aug_ds = train_ds.map(
  lambda x, y: (resize_and_rescale(x, training=True), y))

このアプローチでは、`Dataset.map`を使用して、増強画像のバッチを生成するデータセットを作成します。この場合は、

- データ増強は CPU 上で非同期に行われ、ノンブロッキングです。以下に示すように、`Dataset.prefetch`を使用して GPU 上でのモデルのトレーニングをデータの前処理にオーバーラップさせることができます。
- この場合、`model.save`を呼び出しても、前処理レイヤーはモデルと一緒にエクスポートされません。保存前にモデルに前処理レイヤーをアタッチするか、サーバーサイドで前処理レイヤーを再実装する必要があります。トレーニングの後、エクスポートする前に前処理レイヤーをアタッチすることができます。


1 番目のオプションの例については、[画像分類](https://www.tensorflow.org/tutorials/images/classification)チュートリアルをご覧ください。次に、2 番目のオプションを見てみましょう。

### 前処理レイヤーをデータセットに適用する

上で作成した前処理レイヤーを使用して、トレーニング、検証、テスト用のデータセットを構成します。また、パフォーマンス向上のために、並列読み取りとバッファ付きプリフェッチを使用してデータセットを構成し、I/O がブロックされることなくディスクからバッチを生成できるようにします。データセットのパフォーマンスに関する詳細は、[tf.data API によるパフォーマンス向上](https://www.tensorflow.org/guide/data_performance)ガイドをご覧ください。 

注意: データ増強はトレーニングセットのみに適用されます。

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefecting on all datasets
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

### モデルをトレーニングする

完全性のために、これらのデータセットを使用してモデルをトレーニングします。（目的は仕組みを示すことなので）このモデルには精度のチューニングを行っていません。

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
loss, acc = model.evaluate(test_ds)
print("Accuracy", acc)

### カスタムデータ増強

カスタムデータ増強レイヤーの作成も可能です。このチュートリアルではそのための方法を 2 つ示します。1 つ目は、`layers.Lambda`レイヤーを作成する方法です。これは簡潔なコードの書き込みに適しています。2 つ目は、[サブクラス化](https://www.tensorflow.org/guide/keras/custom_layers_and_models)を介して新しいレイヤーを書き込む方法です。どちらのレイヤーも、確率に従って画像の色をランダムに反転させます。 

In [ ]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) &lt; p:
    x = (255-x)
  else:
    x
  return x

In [ ]:
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = random_invert(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0].numpy().astype("uint8"))
  plt.axis("off")

次に、[サブクラス化](https://www.tensorflow.org/guide/keras/custom_layers_and_models)してカスタムレイヤーを実装します。

In [ ]:
class RandomInvert(layers.Layer):
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor

  def call(self, x):
    return random_invert_img(x)

In [ ]:
_ = plt.imshow(RandomInvert()(image)[0])

どちらのレイヤーも、上記 1 と 2 のオプションで説明した使用が可能です。

## tf.image を使用する

上記の`layers.preprocessing`ユーティリティは便利です。`tf.data`や`tf.image`を使用して独自のデータ増強パイプラインやレイヤーを書き込み、より細かい制御ができます。また、[TensorFlow Addons 画像: 操作](https://www.tensorflow.org/addons/tutorials/image_ops)および [TensorFlow I/O: 色空間の変換](https://www.tensorflow.org/io/tutorials/colorspace)もご覧ください。

花のデータセットは、前にデータ増強で構成したので、再インポートして最初からやり直しましょう。

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

作業に必要な画像を取得します。

In [ ]:
image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

以下の関数を使用して元の画像と増強画像を並べて視覚化し、比較してみましょう。

In [ ]:
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

### データ増強

### 画像を反転する

画像を垂直または水平に反転させます。

In [ ]:
flipped = tf.image.flip_left_right(image)
visualize(image, flipped)

### 画像をグレースケールにする

画像をグレースケールにします。

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
_ = plt.colorbar()

### 画像を飽和させる

飽和係数を指定して画像を飽和させます。

In [ ]:
saturated = tf.image.adjust_saturation(image, 3)
visualize(image, saturated)

### 画像の明るさを変更する

明るさ係数を指定して画像の明るさを変更します。

In [ ]:
bright = tf.image.adjust_brightness(image, 0.4)
visualize(image, bright)

### 画像を中央でトリミングする

画像の中央から好みの部分までをトリミングします。

In [ ]:
cropped = tf.image.central_crop(image, central_fraction=0.5)
visualize(image,cropped)

### 画像を回転させる

画像を 90 度回転させます。

In [ ]:
rotated = tf.image.rot90(image)
visualize(image, rotated)

### データ増強をデータセットに適用する

前に説明したように、`Dataset.map`を使用してデータセットにデータ増強を適用します。

In [ ]:
def resize_and_rescale(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = (image / 255.0)
  return image, label

In [ ]:
def augment(image,label):
  image, label = resize_and_rescale(image, label)
  # Add 6 pixels of padding
  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6) 
   # Random crop back to the original size
  image = tf.image.random_crop(image, size=[IMG_SIZE, IMG_SIZE, 3])
  image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
  image = tf.clip_by_value(image, 0, 1)
  return image, label

### データセットを構成する

In [ ]:
train_ds = (
    train_ds
    .shuffle(1000)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
) 

In [ ]:
val_ds = (
    val_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
test_ds = (
    test_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

前に示したように、これらのデータセットはモデルのトレーニングに使用が可能です。

## 次のステップ

このチュートリアルでは、[Keras 前処理レイヤー](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/)と`tf.image`を使用したデータ増強を実演しました。モデル内に前処理レイヤーを含める方法についての詳細は、[画像分類](https://www.tensorflow.org/tutorials/images/classification)チュートリアルをご覧ください。また、前処理レイヤーがテキスト分類にどのように役立つかについて詳しく知りたい方は、[基本的なテキスト分類](https://www.tensorflow.org/tutorials/keras/text_classification)チュートリアルをご覧ください。`tf.data`に関してはこの[ガイド](https://www.tensorflow.org/guide/data)から、パフォーマンスのために入力パイプラインを構成する方法に関しては[こちら](https://www.tensorflow.org/guide/data_performance)から詳細をご覧ください。